In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk

events = pd.read_csv('events_up_to_01062018.csv')
training = pd.read_csv('labels_training_set.csv')

/home/victorbelosevich/.local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [1]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, r2_score, classification_report, mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [3]:
events.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
training.head()

,person,label
0,0566e9c1,0
1,6ec7ee77,0
2,abe7a2fb,0
3,34728364,0
4,87ed62de,0


In [5]:
eventsWithLabel = events.merge(training, how='left', left_on='person', right_on='person')
eventsWithLabel.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,label
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
eventsWithLabel['timestamp'] = pd.to_datetime(eventsWithLabel['timestamp'])

In [7]:
groupbyPersonas = eventsWithLabel.loc[:,['person', 'event']].groupby('person')
dicc = {}
for clave, data in groupbyPersonas:
    lista = []
    lista = data.drop_duplicates(subset = 'event', keep = 'first')['event'].tolist()
    dicc[clave] = lista

In [8]:
#Obtengo una lista de todos los eventos que tiene esa persona
eventsWithLabel['eventos'] = eventsWithLabel['person'].map(lambda x: dicc[x])
#Obtengo los dias de la semana
eventsWithLabel['dayweek'] = eventsWithLabel.timestamp.dt.dayofweek
#Obtengo la fecha
eventsWithLabel['fecha'] = eventsWithLabel.timestamp.dt.date
#Separo la marca
eventsWithLabel['marca'] = eventsWithLabel['model'].map(lambda x: str(x).split()[0])

In [9]:
#cantidad de personas que ingresaron 27624.
groupbyCantEventos = eventsWithLabel.groupby('person', as_index = False)['event'].agg({'cantEventos': 'count'})
groupbyCantEventos.sort_values(by = 'cantEventos', ascending = False)
eventsWithLabel = pd.merge(eventsWithLabel, groupbyCantEventos, on = 'person')

In [10]:
#Filtro a la gente que no tiene el evento visited_site
personasConVisitas = eventsWithLabel.loc[eventsWithLabel['event'] == 'visited site']\
                             .drop_duplicates(subset = 'person', keep = 'first')['person'].tolist()
dataNotVisit = eventsWithLabel.loc[eventsWithLabel['person'].isin(personasConVisitas)]

In [11]:
eventsWithLabel2 = dataNotVisit.loc[dataNotVisit['event'] == 'visited site'].groupby(['person', 'fecha'], as_index = False).agg({'city': 'first', 'region': 'first', 'country': 'first', 'device_type': 'first', 'operating_system_version': 'first', 'channel': 'first', 'new_vs_returning': 'first'})

In [134]:
eventsWithLabel3 = pd.merge(dataNotVisit, eventsWithLabel2, on = ['person', 'fecha'])
eventsWithLabel3.drop(['region_x', 'device_type_x', 'operating_system_version_x', 'country_x', 'city_x', 'channel_x', 'new_vs_returning_x'], axis = 1, inplace = True)
eventsWithLabel3.rename(columns = {'region_y': 'region', 'device_type_y': 'device_type', 'operating_system_version_y': 'SO', 'city_y': 'city', 'country_y': 'country', 'channel_y': 'channel', 'new_vs_returning_y': 'new_vs_returning'}, inplace =True)

In [135]:
def obtenerSistema(so):
    sistema = so.split()
    if len(sistema) > 1:
        otro = sistema[1].split('.')[0]
        return sistema[0] + ' ' + otro
    else:
        return sistema[0]

In [136]:
eventsWithLabel3['SO'] = eventsWithLabel3['SO'].map(obtenerSistema)

In [137]:
eventsWithLabel3.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,fecha,marca,cantEventos,city,SO,country,region,device_type,channel,new_vs_returning
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,2018-05-18,Samsung,9,Cantagalo,Android 7,Brazil,Rio de Janeiro,Smartphone,Organic,New
1,2018-05-18 00:30:30,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,2018-05-18,Samsung,9,Cantagalo,Android 7,Brazil,Rio de Janeiro,Smartphone,Organic,New
2,2018-05-18 00:07:23,search engine hit,4886f805,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2018-05-18,nan,9,Cantagalo,Android 7,Brazil,Rio de Janeiro,Smartphone,Organic,New
3,2018-05-18 00:11:56,checkout,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,2018-05-18,Samsung,9,Cantagalo,Android 7,Brazil,Rio de Janeiro,Smartphone,Organic,New
4,2018-05-18 00:11:35,viewed product,4886f805,NaN,9287.0,Samsung Galaxy J7 Prime,Muito Bom,32GB,Dourado,NaN,...,2018-05-18,Samsung,9,Cantagalo,Android 7,Brazil,Rio de Janeiro,Smartphone,Organic,New


In [139]:
testDF = eventsWithLabel3.loc[eventsWithLabel3['label'].isnull()]
testDF.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,fecha,marca,cantEventos,city,SO,country,region,device_type,channel,new_vs_returning
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,2018-05-18,Samsung,9,Cantagalo,Android 7,Brazil,Rio de Janeiro,Smartphone,Organic,New
1,2018-05-18 00:30:30,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,2018-05-18,Samsung,9,Cantagalo,Android 7,Brazil,Rio de Janeiro,Smartphone,Organic,New
2,2018-05-18 00:07:23,search engine hit,4886f805,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2018-05-18,nan,9,Cantagalo,Android 7,Brazil,Rio de Janeiro,Smartphone,Organic,New
3,2018-05-18 00:11:56,checkout,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,2018-05-18,Samsung,9,Cantagalo,Android 7,Brazil,Rio de Janeiro,Smartphone,Organic,New
4,2018-05-18 00:11:35,viewed product,4886f805,NaN,9287.0,Samsung Galaxy J7 Prime,Muito Bom,32GB,Dourado,NaN,...,2018-05-18,Samsung,9,Cantagalo,Android 7,Brazil,Rio de Janeiro,Smartphone,Organic,New


In [138]:
trainingDF = eventsWithLabel3.loc[eventsWithLabel3['label'].notnull()]
trainingDF.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,fecha,marca,cantEventos,city,SO,country,region,device_type,channel,new_vs_returning
9,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,2018-05-18,iPhone,65,São Paulo,Android 5,Brazil,Sao Paulo,Smartphone,Paid,Returning
10,2018-05-18 00:23:33,viewed product,ad93850f,NaN,318.0,iPhone 5s,Muito Bom,64GB,Prateado,NaN,...,2018-05-18,iPhone,65,São Paulo,Android 5,Brazil,Sao Paulo,Smartphone,Paid,Returning
11,2018-05-18 00:16:10,viewed product,ad93850f,NaN,5907.0,iPhone 5s,Bom - Sem Touch ID,16GB,Cinza espacial,NaN,...,2018-05-18,iPhone,65,São Paulo,Android 5,Brazil,Sao Paulo,Smartphone,Paid,Returning
12,2018-05-18 00:14:55,viewed product,ad93850f,NaN,6023.0,iPhone 5s,Bom - Sem Touch ID,16GB,Dourado,NaN,...,2018-05-18,iPhone,65,São Paulo,Android 5,Brazil,Sao Paulo,Smartphone,Paid,Returning
13,2018-05-18 00:11:26,ad campaign hit,ad93850f,/comprar/iphone/iphone-5s,NaN,NaN,NaN,NaN,NaN,NaN,...,2018-05-18,nan,65,São Paulo,Android 5,Brazil,Sao Paulo,Smartphone,Paid,Returning


In [140]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [141]:
eventosUnicos = eventsWithLabel3.drop_duplicates(subset="event", keep="first").reset_index()["event"]

In [142]:
label_encoder = LabelEncoder()
label_encoder.fit(eventosUnicos)
integer_encoded = label_encoder.transform(eventsWithLabel3["event"])

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)


In [143]:
for i, event in enumerate(eventosUnicos):
    eventsWithLabel3[event] = onehot_encoded[:, label_encoder.transform(data)[i]]

viewed product
search engine hit
checkout
searched products
generic listing
visited site
ad campaign hit
brand listing
lead
staticpage
conversion


In [151]:
eventsWithLabel3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2319285 entries, 0 to 2319284
Data columns (total 39 columns):
timestamp            datetime64[ns]
event                object
person               object
url                  object
sku                  float64
model                object
condition            object
storage              object
color                object
skus                 object
search_term          object
staticpage           float64
campaign_source      object
search_engine        object
screen_resolution    object
browser_version      object
label                float64
eventos              object
dayweek              int64
fecha                object
marca                object
cantEventos          int64
city                 object
SO                   object
country              object
region               object
device_type          object
channel              object
new_vs_returning     object
viewed product       float64
search engine hit    float64
checkout

In [145]:
def logisticReg(x_train, x_test, y_train, y_test):
    logReg = LogisticRegression()
    logReg.fit(x_train, y_train)
    y_pred = logReg.predict(x_test)
    return np.sqrt(mean_squared_error(y_test, y_pred))
#TAMBIEN PUEDEN USAR PARA VER EL ACCURACY
#ACCURACY = accuracy_score(y_test, y_pred)

In [146]:
def knn(k, x_train, x_test, y_train, y_test):
    knn = KNeighborsClassifier(n_neighbors=k, metric='manhattan')
    knn.fit(x_train, y_train)
    y_pred = knn.predict(x_test)
    return np.sqrt(mean_squared_error(y_test, y_pred))
#TAMBIEN PUEDEN USAR PARA VER EL ACCURACY
#ACCURACY = accuracy_score(y_test, y_pred)

In [147]:
def decisionTree(max_depth, max_features, x_train, x_test, y_train, y_test):
    dt = DecisionTreeClassifier(max_depth=max_depth, max_features=max_features)
    dt.fit(x_train, y_train)
    y_pred = dt.predict(x_test)
    return np.sqrt(mean_squared_error(y_test, y_pred))
#TAMBIEN PUEDEN USAR PARA VER EL ACCURACY
#ACCURACY = accuracy_score(y_test, y_pred)

In [148]:
def xgboost(max_depth, n_estimators, x_train, x_test, y_train, y_test):
    xgb = XGBClassifier(max_depth=max_depth, n_estimators=n_estimators)
    xgb.fit(x_train, y_train)
    y_pred = xgb.predict(x_test)
    return np.sqrt(mean_squared_error(y_test, y_pred))
#TAMBIEN PUEDEN USAR PARA VER EL ACCURACY
#ACCURACY = accuracy_score(y_test, y_pred)